# SustainKieker Hackathon: Reverse Engineering of Research Software

🗓️ Thu 27/02, 13:30
<br>📍 Room SR 206, Building 30.70, Campus South (KIT)
<br>ℹ️ [Abstract](https://events.hifis.net/event/1741/contributions/14031/)
<br>🔬 [The Kieker Observability Framework](https://kieker-monitoring.net)
<br>🌐 [The SustainKieker Project Homepage](https://sustainkieker.kieker-monitoring.net)

## Table of Contents

1. Introduction
1. Prerequisites
   * Linux system
   * Software
1. Instructions
1. Run the Kieker Trace Analysis
1. More reads

## Introduction

[Kieker observability framework](https://kieker-monitoring.net/) features monitoring and analysis capabilities.
OpenTelemetry, in comparison, provides means to monitor the program but analysis. Our new [Otkt DSL](https://github.com/kieker-monitoring/OtktDSL) can define a mapping from an OpenTelemetry Span to a Kieker record.
Using Otkt, we show how we can use OpenTelemetry to collect monitoring data from a Python program and send it to a Kieker Analysis endpoint.

## Prerequisites

### Linux system
   
#### macOS
* [Multipass](https://github.com/canonical/multipass) ([latest download](https://canonical.com/multipass/download/macos))
  - [x] M1, M2, or Intel based
  - [x] macOS 10.15 Catalina or later

#### Windows 10/11 (two options)
* Multipass ([latest download](https://canonical.com/multipass/download/windows))
  - [x] Windows 10 Pro or Enterprise version 1803 (“April 2018 Update”) or later
* WSL Install Instructions
  * [Windows 10 2004 or higher](https://learn.microsoft.com/en-us/windows/wsl/install)
  * Old versions: [Windows 10 1903 or higher](https://learn.microsoft.com/en-us/windows/wsl/install-manual)

### Software
1. Python3

1. Java
   ```bash
   # Fedora 40, 41, 42
   sudo dnf install java-21-openjdk-devel
   # Ubuntu 20.04, 22.04, 24.04, 24.10
   sudo apt install openjdk-21-jdk
   ```
1. Maven
   ```bash
   # Fedora 40, 41, 42
   sudo dnf install maven
   # Ubuntu 20.04, 22.04, 24.04, 24.10
   sudo apt install maven
   ```
   or [install manually](https://maven.apache.org/install.html).

1. [GNU plotutils](http://www.gnu.org/software/plotutils/)
   ```bash
   # Fedora 40, 41, 42
   sudo dnf install plotutils
   # Ubuntu 20.04, 22.04, 24.04, 24.10
   sudo apt install plotutils
   ```
1. The [graphviz](http://www.graphviz.org/) graph visualization tools
   ```bash
   # Fedora 40, 41, 42
   sudo dnf install graphviz
   # Ubuntu 20.04, 22.04, 24.04, 24.10
   sudo apt install graphviz
   ```

1. The [Ghostscript](https://ghostscript.com/) interpreter for PostScript language & PDF
   ```bash
   # Fedora 40, 41, 42
   sudo dnf install ghostscript
   # Ubuntu 20.04, 22.04, 24.04, 24.10
   sudo apt install ghostscript
   ```

---
## Instructions

### Build the Otkt collector

* It receives the monitoring data.
* **Already done it the Julia container.**

  ```shell
  cd ~/Hackathon/collector
  mvn clean package
  ```

### Export the `PYTHONPATH` variable

* The `otkt` python package should be visible.
* **Already done in the Julia container.**

  ```bash
  export PYTHONPATH=/path/to/hackathon-repo/python:$PYTHONPATH
  ```

### Install Python dependencies

* **Already done in the Julia container.**
```bash
pip install -r ~/Hackathon/res/requirements.txt
```

---
## Instrumenting Python code

* Manual instrumentation requires locating\
  (1) target python files\
  (2) the file with the entry point.
* Please ask for help if something does not work.

### Instrument the entry point

```python
from otkt.otelinit import tracer
```

### Instrument all python files

```bash
# Run the below command inside the target software repository:
~/Hackathon/tools/instrument-py.sh
```

### Run the Otkt collector

In [ ]:
!java -jar ~/Hackathon/collector/target/Collector-0.0.1-SNAPSHOT-jar-with-dependencies.jar -c ~/Hackathon/res/config.txt

13:56:06.803 [main] DEBUG Main -- Collector
13:56:06.832 [main] DEBUG kieker.analysis.generic.sink.DataSink:DataSink-0 -- numOpenedInputPorts (inc): 1
13:56:06.832 [main] DEBUG kieker.analysis.generic.sink.DataSink -- Configuration complete.
13:56:06.841 [main] INFO kieker.monitoring.core.controller.TCPController -- Could not parse port for the TCPController, deactivating this option. Received string was: 
13:56:06.904 [main] WARN kieker.common.configuration.Configuration -- Error parsing configuration property 'kieker.monitoring.writer.filesystem.FileWriter.bufferSize' of type int, found value '81920 ', using default value 65536
13:56:06.914 [main] DEBUG kieker.monitoring.core.controller.WriterController -- Initializing Writer Controller
13:56:06.914 [main] INFO kieker.monitoring.core.controller.StateController -- Enabling monitoring
13:56:06.914 [Thread-0] DEBUG kieker.monitoring.writer.MonitoringWriterThread -- kieker.monitoring.writer.MonitoringWriterThread is running.
13:56:06.922

In [ ]:
from otkt.otelinit import tracer
from uxsim import World

# Define the main simulation
# Units are standardized to seconds (s) and meters (m)
W = World(
    name="",    # Scenario name
    deltan=5,   # Simulation aggregation unit delta n
    tmax=1200,  # Total simulation time (s)
    print_mode=1, save_mode=1, show_mode=1,    # Various options
    random_seed=0    # Set the random seed
)

# Define the scenario
## Create nodes
W.addNode(name="orig1", x=0, y=0)  #xy coords are for visualization 
W.addNode(name="orig2", x=0, y=2)
W.addNode(name="merge", x=1, y=1)
W.addNode(name="dest", x=2, y=1)
## Create links between nodes
W.addLink(name="link1", start_node="orig1", end_node="merge",
          length=1000, free_flow_speed=20, number_of_lanes=1)
W.addLink(name="link2", start_node="orig2", end_node="merge", 
          length=1000, free_flow_speed=20, number_of_lanes=1)
W.addLink(name="link3", start_node="merge", end_node="dest", 
          length=1000, free_flow_speed=20, number_of_lanes=1)
## Create OD traffic demand between nodes
W.adddemand(orig="orig1", dest="dest", t_start=0, t_end=1000, flow=0.45)
W.adddemand(orig="orig2", dest="dest", t_start=400, t_end=1000, flow=0.6)

# Run the simulation to the end
W.exec_simulation()

# Print summary of simulation result
W.analyzer.print_simple_stats()

# Visualize snapshots of network traffic state for several timesteps
W.analyzer.network(t=100, detailed=1, network_font_size=12)
W.analyzer.network(t=600, detailed=1, network_font_size=12)
W.analyzer.network(t=800, detailed=1, network_font_size=12)

1. On a new terminal, run the target program (e.g., `python3 main.py`).

    ```bash
    # Run inside the target software repository:
    python3 main.py
    ```

## Run the Kieker Trace Analysis

   * Prepare Trace Analysis
     ```bash
     # Run inside the hackathon repository
     unzip tools/trace-analysis-2.0.2.zip
     ```

   * Aggregated Deployment Call Tree
     ```bash
     # Run inside the hackathon repository
     mkdir output-adctree
     trace-analysis-2.0.2/bin/trace-analysis  \
       --inputdirs /path/to/kieker-logs       \
       --outputdir output-adctree             \
       --plot-Aggregated-Deployment-Call-Tree
     cd output-adctree
     dot aggregatedDeploymentCallTree.dot -T pdf output.pdf
     ```

   * Deployment Component Dependency Graph
     ```bash
     # Run inside the hackathon repository
     mkdir output-dcdgraph
     trace-analysis-2.0.2/bin/trace-analysis             \
       --inputdirs /path/to/kieker-logs                  \
       --outputdir output-dcdgraph
       --plot-Deployment-Component-Dependency-Graph none
     cd output-dcdgraph
     dot deploymentComponentDependencyGraph.dot -T pdf output.pdf
     ```

   * Deployment Sequence Diagrams
     ```bash
     # Run inside the hackathon repository
     mkdir output-dsdiagrams
     trace-analysis-2.0.2/bin/trace-analysis             \
       --inputdirs /path/to/kieker-logs                  \
       --outputdir output-dsdiagrams
       --plot-Deployment-Sequence-Diagrams
     cd output-dsdiagrams
     pic2plot -T ps deploymentSequenceDiagram-1.pic > output.ps
     ps2pdf outout.ps output.pdf
     ```

## More Reads
1. The Otkt collector stores all received records under `/tmp`. E.g.,

   ```
   /tmp/kieker-20250217-181132-41826294550971-UTC--KIEKER/
   /tmp/kieker-20250218-093839-12420615078112-UTC--KIEKER/
   /tmp/kieker-20250218-114626-20087904425947-UTC--KIEKER/
   ```

   You can change the location by changing the value for
   `kieker.monitoring.writer.filesystem.FileWriter.customStoragePath` in
   [res/config.txt](res/config.txt).

1. Packaging a python software

   Instrumenting a Python software may require downloading the source code and
building a package. In this case, follow the instruction below and repackage it
yourself. It could be as simple as running `python3 -m build`. Otherwise, you
need to follow the provided build instruction.

   ```bash
   # Instrument the target software before running the below commands
   pip install build
   python3 -m build
   ```

1. Building an Otkt program

   For the Hackathon, we provide all artifacts from the Otkt build of the Otkt source code [demo.otkt](examples/demo.otkt). For more details, refer to [BUILD.md](BUILD.md).

1. Building the Otkt compiler

   The Otkt compiler is maintained in the GitHub repository [OtktDSL](https://github.com/kieker-monitoring/OtktDSL). The instruction will be uploaded soon.